In [ ]:
# This block of code is used to add the root folder of the project to the path so that src can be imported.
import os
import sys

root_folder = os.path.dirname(os.path.abspath(""))
if not root_folder in sys.path:
    sys.path.append(root_folder)

In [ ]:
import os
from pathlib import Path

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import wandb

from src import read_trips, process_trips, create_pipeline, save_model

In [ ]:
PROJECT_NAME = "tip-prediction-random-forest"
MODEL_NAME = f"{PROJECT_NAME}-model"

DATA_DIR = Path("../data")
MODEL_DIR = Path("../models")

# Load data

In [ ]:
trips_train = read_trips(DATA_DIR, color="green", year="2022", month="1")
trips_val = read_trips(DATA_DIR, color="green", year="2022", month="2")
trips_test = read_trips(DATA_DIR, color="green", year="2022", month="3")

trips_train = process_trips(trips_train)
trips_val = process_trips(trips_val)
trips_test = process_trips(trips_test)

target = "tip_amount"
categorical_cols = ["PU_DO"]
numerical_cols = ["trip_distance"]
used_cols = categorical_cols + numerical_cols

X_train = trips_train[used_cols]
y_train = trips_train[target]

X_val = trips_val[used_cols]
y_val = trips_val[target]

# Using Wandb to track experiments

1. Before starting, make sure you have a wandb account and log in to it by API key.
2. Create a new wandb run by using `wandb.init()`.
3. To log an artifact, create an Artifact object by `wandb.Artifact()`, and add the artifact by `add_file()`. Then log the artifact by `wandb.log_artifact()`.
4. During training, log the metrics you want to track by using `wandb.log()`. The input should be a dictionary.

In [ ]:
wandb.login()

In [ ]:
wandb.init(project=PROJECT_NAME, job_type="train")

In [ ]:
train_artifact = wandb.Artifact("green_2022_01", type="train_data")
train_artifact.add_file(DATA_DIR / "green_tripdata_2022-01.parquet")
wandb.run.log_artifact(train_artifact)

val_artifact = wandb.Artifact("green_2022_02", type="val_data")
val_artifact.add_file(DATA_DIR / "green_tripdata_2022-02.parquet")
wandb.run.log_artifact(val_artifact)

pipe = create_pipeline(RandomForestRegressor(max_depth=10, random_state=0))
pipe.fit(X_train, y_train)

rmse = mean_squared_error(y_val, pipe.predict(X_val), squared=False)
wandb.log({"RMSE": rmse})

save_model(MODEL_DIR, "rf_predictor.pkl", pipe)

artifact = wandb.Artifact(MODEL_NAME, type="model")
artifact.add_file(MODEL_DIR / "rf_predictor.pkl")
wandb.log_artifact(artifact)

# Tuning hyperparameters by sweep

The step to tune hyperparameters by sweep is as follows:

1. Wrap the training code in a function.
2. Create a search space of hyperparameters.
3. Create a sweep id by `wandb.sweep()` with the search space and project name.
4. Run the sweep by `wandb.agent()` with the sweep id and the function created in step 1.

In [ ]:
def run_train():
    wandb.init()
    config = wandb.config

    train_artifact = wandb.Artifact("green_2022_01", type="train_data")
    train_artifact.add_file(DATA_DIR / "green_tripdata_2022-01.parquet")
    wandb.run.log_artifact(train_artifact)

    val_artifact = wandb.Artifact("green_2022_02", type="val_data")
    val_artifact.add_file(DATA_DIR / "green_tripdata_2022-02.parquet")
    wandb.run.log_artifact(val_artifact)

    pipe = create_pipeline(RandomForestRegressor(**config, random_state=0))
    pipe.fit(X_train, y_train)

    rmse = mean_squared_error(y_val, pipe.predict(X_val), squared=False)
    wandb.log({"rmse_val": rmse})

    save_model(MODEL_DIR, "rf_predictor.pkl", pipe)

    artifact = wandb.Artifact(MODEL_NAME, type="model")
    artifact.add_file(MODEL_DIR / "rf_predictor.pkl")
    wandb.log_artifact(artifact)

sweep_config = {
    "method": "bayes",
    "metric": {"name": "rmse_val", "goal": "minimize"},
    "parameters": {
        "max_depth": {
            "distribution": "int_uniform",
            "min": 1,
            "max": 20,
        },
        "n_estimators": {
            "distribution": "int_uniform",
            "min": 10,
            "max": 50,
        },
        "min_samples_split": {
            "distribution": "int_uniform",
            "min": 2,
            "max": 10,
        },
        "min_samples_leaf": {
            "distribution": "int_uniform",
            "min": 1,
            "max": 4,
        },
    },
}

sweep_id = wandb.sweep(sweep_config, project=PROJECT_NAME)
wandb.agent(sweep_id, function=run_train, count=5)

#  Registering the best model from the sweep to the model registry

1. Use `wandb.Api()` to request the information of the sweep.
2. Sort the runs by the metric you want to use to select the best model. And the first run is the best model.
3. Use `run.logger_artifact()` to get the path of the model artifact on wandb server.
4. Use wandb.Api().artifacts() to get the artifact object and link it to the model registry by `link()`.

In [ ]:
api = wandb.Api()
sweep = api.sweep(f"shunlungchang/tip-prediction-random-forest/{sweep_id}")
runs = sorted(sweep.runs, key=lambda run: run.summary.get("rmse_val", 0))

In [ ]:
for artifact in runs[0].logged_artifacts():
    if artifact.type == "model":
        artifact_name = artifact.name
        break

In [ ]:
artifact = wandb.Api().artifact(f"{runs[0].entity}/{runs[0].project}/{artifact_name}")
artifact.link(f"{runs[0].entity}/{runs[0].project}/tip-predictor", aliases=["latest"])